### Choose a Georgian song and play all three parts

### Get Data.

#### Create an html parser to read the directory manifest

In [ ]:
from html.parser import HTMLParser
from os.path import basename

new_track = False
class MyHTMLParser(HTMLParser):
    def __init__ (self):
        HTMLParser.__init__(self)
        self.data = []

    def handle_starttag(self, tag, attrs):
        new_track = True
        if tag == "track-wrap":
            for key, val in attrs:
                if key == "title":
                    self.data.append(val)
        if tag == "video":
            self.data.append("video")
        if tag == "source":
            for key, val in attrs:
                if key == "src":
                    self.data.append(basename(val))

    def handle_endtag(self, tag):
        if tag == "track-wrap" or tag == "video":
            new_track = False

    def handle_data(self, data):
        pass
    
    def get_data (self):
        return zip(self.data, self.data[1:] + self.data[:1])



### Get all the files in one Collection/Song directory.

In [ ]:
import librosa
import re
from os import listdir
from os.path import isdir, join, splitext, exists, dirname

def getFiles(coll, sd, wav=False, debug=0):
    files = dict()
    if coll == "Scherbaum Mshavanadze":
        html= open(join(root, coll, sd, 'page.txt')).read()
        parser = MyHTMLParser()
        parser.feed(html)
        files['throat'] = [f[1] for f in \
                           list(filter(lambda x: re.search("Throat microphone [0-9]+", x[0]), parser.get_data()))]
        files['headset'] = [f[1] for f in \
                            list(filter(lambda x: re.search("Headset microphone [0-9]+", x[0]), parser.get_data()))]
        
        # For S-M, bass = 3, top = 1
        files['throat'].reverse()
        files['headset'].reverse()
    else:
        suffixes = dict()
        suffixes["Teach Yourself Megrelian Songs"] = [" (bass)", " (middle)", " (top)"]
        suffixes["Teach Yourself Gurian Songs"] = [" - bass", " - middle", " - top"]
        files['headset'] = [sd + suff + ".mp3" for suff in suffixes[coll]]

        if coll == "Teach Yourself Megrelian Songs":
            sd = "mp3"

    return (sd, files)
    


In [ ]:
%%capture --no-stdout --no-display
collections = {"sm":"Scherbaum Mshavanadze",
               "guria":"Teach Yourself Gurian Songs",
               "megrelia":"Teach Yourself Megrelian Songs"}

working_coll = collections["sm"]
working_song = "GVM017_ChvenMshvidobaTake2_Ozurgeti_ShalvaChemo2016_20160713"

root = '/Akamai/voice/data/'
for coll in collections.values():
    print("\n", coll, ":")
    songdirs = [d for d in listdir(join(root,coll)) \
     if isdir(join(root, coll, d)) and re.match("[A-Z]+", d) != None]
    for sd in songdirs: 
        if working_song and sd != working_song:
            continue
        debug = 1        
        # one collection has its mp3s in a different sd
        (sd_data, files) = getFiles(coll, sd, debug)
        
        f = files['headset'][0]
        data_path = join(root, coll, sd_data, f)
        y, sr = librosa.load(data_path, sr=None)
        print(f'  {sd}: {y.size/sr:.2f} seconds.')


In [ ]:
%%capture --no-stdout --no-display
%matplotlib inline
import IPython.display as ipd

# Listen to song

coll = working_coll
song = working_song
debug = 0
(sd, files) = getFiles(coll, song, debug)
k = 0
for f in files['headset']:
    if k<2:
        k=k+1
        #continue
    data_path = join(root, coll, sd, f)
    y, sr = librosa.load(data_path, sr=None)
    start_sample = 0*sr
    end_sample = y.size
    print(f)
    ipd.display(ipd.Audio(y[start_sample:end_sample], rate=sr))
print("done")

